In [ ]:
# TODO: 
# - Estimate center of diabolo from data without manual calibration:
#   - Find range of frames where absolute motion of diabolo is small
#   - Find rotation axis in that frame range
#   - Place the rotation axis on the average position, move backwards along the axis, and set that as the center 
#        (the distance for this step may be taken from the manual calibration)


# Then:
# Make a function that for a given DataFrame:
# - Calculates the rotation axis
# - Calculates the rotation speed (around that axis) for each frame
# - Plots the rotation speed and diabolo position
# - Calculates the offset from the rigid body center to the rotation axis

In [7]:
# EXAMPLE USAGE
c = ExperimentClass()
experiment_directory = "../experiments/recordings/2020-04-05/"
output_directory = "../experiments/output/2020-04-05/"
c.smooth_and_offset_data(experiment_filename=experiment_directory+"Take_diabolo_red.csv", 
                         output_filename=output_directory+"Take_diabolo_red_smoothed.csv", 
                         calibration_file_directory=experiment_directory)

Calculating offsets
Applying offsets
Applying offsets (this may take time)
Applying offsets (this may take time)
Applying offsets (this may take time)
Done applying offsets


In [2]:
import pandas as pd
import numpy as np
import copy
import math
import os
import sys
import transformations as tftf

In [3]:
def angle_between_quaternions(q1, q2):
    """Taken from https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation#Recovering_the_axis-angle_representation
    """
    q2_to_1 = tftf.quaternion_multiply(tftf.quaternion_inverse(q1), q2)
    angle = 2*math.atan2(np.linalg.norm(q2_to_1[0:3]), q2_to_1[3])
    return angle

In [4]:
class ExperimentClass():
  """
  Used for diabolo data treatment
  """

  def __init__(self):
    self.DEBUG_DISPLAY = False
    
  def read_raw_files(self, filename="../experiments/recordings/2020-04-05/Take_diabolo_red.csv"):
    print("Reading experiment file")
    self.experiment_df = pd.read_csv(filename, header=[2,4,5])
    self.diabolo_data = self.experiment_df["diabolo_red"]
    self.stick_left_data = self.experiment_df["stick_left"]
    self.stick_right_data = self.experiment_df["stick_right"]
    if self.DEBUG_DISPLAY:
      print("diabolo_data.head() = ")
      print(self.diabolo_data.head())
      print("stick_left.head() = ")
      print(self.stick_left_data.head())

  def calculate_and_apply_offsets(self, experiment_directory="../experiments/recording/2020-04-05"):
    self._get_offsets_from_files(experiment_directory)

    if self.DEBUG_DISPLAY:
      self.diabolo_data_original = copy.deepcopy(self.experiment_df["diabolo_red"])
      self.stick_left_data_original = copy.deepcopy(self.experiment_df["stick_left"])
      self.stick_right_data_original = copy.deepcopy(self.experiment_df["stick_right"])
      self.apply_offsets_to_pose_df(self.experiment_df, "diabolo_red", self.translation_diabolo_intermediate, self.rotation_diabolo_intermediate)
      self.diabolo_data_intermediate = self.experiment_df["diabolo_red"]
      self.read_raw_files()

    print("Applying offsets")
    self.apply_offsets_to_pose_df(self.experiment_df, "diabolo_red", self.translation_diabolo_red_to_diabolo_red_center, self.rotation_diabolo_red_to_diabolo_red_center)
    self.apply_offsets_to_pose_df(self.experiment_df, "stick_left", self.translation_left_stick_to_tip)
    self.apply_offsets_to_pose_df(self.experiment_df, "stick_right", self.translation_right_stick_to_tip)
    print("Done applying offsets")
    self.diabolo_data = self.experiment_df["diabolo_red"]
    self.stick_left_data = self.experiment_df["stick_left"]
    self.stick_right_data = self.experiment_df["stick_right"]
    if self.DEBUG_DISPLAY:
      print("diabolo_data.head() = ")
      print(self.diabolo_data.head())
      print("stick_left.head() = ")
      print(self.stick_left_data.head())
        
  def _get_offsets_from_files(self, experiment_directory="../experiments/recording/2020-04-05", rotated_calibration_coordinate_system=True):
    """
    Get the offsets for each rigid body. The offset transforms from the center of the rigid body
    to the point of interest (stick tip or diabolo center).
    """
    print("Calculating offsets")
    f = pd.read_csv(os.path.join(experiment_directory, "diabolo_calib_red_2020-05-25_full.csv"), header=[2,4,5])

    if rotated_calibration_coordinate_system:
      ### === 
      # Inconveniently, the calibration data on 2020-05 was recorded in a coordinate frame that was not the same as during the experiments.
      # This block compensates for this by rotating the calibration data.
      
      translation_diabolo_red = f["diabolo_red"]["Position"].iloc[1].values   # 
      rotation_diabolo_red = tftf.quaternion_inverse(f["diabolo_red"]["Rotation"].iloc[1].values)
      q0 = tftf.quaternion_from_euler(0, 0, -90.0/180.0*math.pi)
      R0 = tftf.quaternion_matrix(q0)
      # print("q0 = ", q0)
      translation_diabolo_red = np.array([translation_diabolo_red[0], translation_diabolo_red[1], translation_diabolo_red[2], 1])
      # print("translation_diabolo_red before rotation", translation_diabolo_red)
      translation_diabolo_red = np.dot(R0, translation_diabolo_red)
      # print("translation_diabolo_red after rotation", translation_diabolo_red)
      # print("rotation_diabolo_red before rotation", rotation_diabolo_red)
      # rotation_diabolo_red = tftf.quaternion_multiply(rotation_diabolo_red, q0)
      rotation_diabolo_red = tftf.quaternion_multiply(rotation_diabolo_red, tftf.quaternion_inverse(q0))
      # print("rotation_diabolo_red after rotation", rotation_diabolo_red)
      ### ===

    # The transformation from the diabolo motion capture frame to its center is calculated by adding:
    # 1. The transformation from the motion capture frame to the origin
    # 2. The transformation from the origin to the diabolo center (which is known from the controlled placement during calibration)
    translation_diabolo_red_to_world = -translation_diabolo_red[0:3]
    rotation_diabolo_red_to_world = tftf.quaternion_inverse(rotation_diabolo_red)
    
    # 1. From diabolo red motion capture frame to world
    T1 = tftf.translation_matrix(translation_diabolo_red_to_world)
    R1 = tftf.quaternion_matrix(rotation_diabolo_red_to_world)
    M1_diabolo_red_to_world = tftf.concatenate_matrices(T1, R1)
    
    # 2. From world to diabolo center
    T2 = tftf.translation_matrix([0, 0, 0.0722])
    R2 = tftf.euler_matrix(0, 0, 0/180*math.pi)
    M2_world_to_diabolo_red_center = tftf.concatenate_matrices(T2, R2)
    
    M_res = np.dot(M1_diabolo_red_to_world, M2_world_to_diabolo_red_center)
    
    self.rotation_diabolo_red_to_diabolo_red_center = tftf.quaternion_from_matrix(M_res)
    self.translation_diabolo_red_to_diabolo_red_center = tftf.translation_from_matrix(M_res)

    # Obtain the stick positions. The tip of the stick was put at the origin during calibration, 
    # so the transformation to the origin is the translation to the tip.
    f = pd.read_csv(os.path.join(experiment_directory, "stick_left_calib_2020-05-26_full.csv"), header=[2,4,5])
    self.translation_left_stick_to_tip = -f["stick_left"]["Position"].iloc[1].values
    self.translation_left_stick_to_tip[2] = self.translation_left_stick_to_tip[2] + 0.004  # The small offset is due to the thickness of the stick

    f = pd.read_csv(os.path.join(experiment_directory, "stick_right_calib_2020-05-26_full.csv"), header=[2,4,5])
    self.translation_right_stick_to_tip = -f["stick_right"]["Position"].iloc[1].values 
    self.translation_right_stick_to_tip[2] = self.translation_right_stick_to_tip[2] + 0.004

    if rotated_calibration_coordinate_system:
      ### === See above
      test = copy.deepcopy(self.translation_left_stick_to_tip)
      self.translation_left_stick_to_tip[0] = test[1]
      self.translation_left_stick_to_tip[1] = -test[0]    
      t2 = copy.deepcopy(self.translation_right_stick_to_tip)
      self.translation_right_stick_to_tip[0] = t2[1]
      self.translation_right_stick_to_tip[1] = -t2[0]    
      ### ===

    if self.DEBUG_DISPLAY:
      print("self.translation_diabolo_red_to_diabolo_red_center = ")
      print(self.translation_diabolo_red_to_diabolo_red_center)
      print("self.rotation_diabolo_red_to_diabolo_red_center = ")
      print(self.rotation_diabolo_red_to_diabolo_red_center)
      print("self.translation_left_stick_to_tip = ")
      print(self.translation_left_stick_to_tip)
      print("self.translation_right_stick_to_tip = ")
      print(self.translation_right_stick_to_tip)
      self.rotation_diabolo_intermediate = tftf.quaternion_from_matrix(M1_diabolo_red_to_world)
      self.translation_diabolo_intermediate = tftf.translation_from_matrix(M1_diabolo_red_to_world)
        
  def write_offset_files(self, filename="experiments/output/2020-04-05/Take_diabolo_red.csv"):
    try:
      head, tail = os.path.split(filename)
      os.makedirs(head)
    except:
      pass
    self.experiment_df.to_csv(filename)
    print("Finished writing offset files to ", filename)

  def apply_offsets_to_pose_df(self, df, rigid_body_name, translation_offset, rotation_offset = []):
    print("Applying offsets (this may take time)")
    for i in range(len(df)):
      if len(translation_offset):
        p = df.loc[i, (rigid_body_name, "Position")].values
        p_new = p + translation_offset
        df.at[i, (rigid_body_name, "Position", "X")] = p_new[0]
        df.at[i, (rigid_body_name, "Position", "Y")] = p_new[1]
        df.at[i, (rigid_body_name, "Position", "Z")] = p_new[2]
      if len(rotation_offset):
        quat = df.loc[i, (rigid_body_name, "Rotation")].values
        q_new = tftf.quaternion_multiply(tftf.unit_vector(quat), rotation_offset)
        df.at[i, (rigid_body_name, "Rotation", "X")] = q_new[0]
        df.at[i, (rigid_body_name, "Rotation", "Y")] = q_new[1]
        df.at[i, (rigid_body_name, "Rotation", "Z")] = q_new[2]
        df.at[i, (rigid_body_name, "Rotation", "W")] = q_new[3]
  
  def smooth_and_offset_data(self, experiment_filename="../experiments/recordings/2020-04-05/Take_diabolo_red.csv", 
                      output_filename="../experiments/output/2020-04-05/Take_diabolo_red_smoothed.csv", 
                      calibration_file_directory="../experiments/recordings/2020-04-05/", 
                      smooth_frame_window=15):
    """Read in, smooth position data and offset the data using calibration files.
    """
    f = pd.read_csv(experiment_filename, header=[2,4,5])
    f2 = f.copy()
    f_smoothed = f.rolling(15).mean()   # 5 frames is still wobbly
    f2.loc[:, ("diabolo_red","Position")] = f_smoothed
    f2.loc[:, ("stick_left","Position")] = f_smoothed
    f2.loc[:, ("stick_right","Position")] = f_smoothed
    c.experiment_df = f2

    c.diabolo_data = c.experiment_df["diabolo_red"]
    c.stick_left_data = c.experiment_df["stick_left"]
    c.stick_right_data = c.experiment_df["stick_right"]

    c.calculate_and_apply_offsets(experiment_directory)
    c.write_offset_files(output_filename)

In [5]:
def estimate_rotation_axis(experiment_df, start_index, amount_of_frames):
    """Estimate the rotation axis from a series of orientations in a DataFrame.
    To get the rotation axis, we need to minimize the distance between 
    the resulting vectors of the rotation axis multiplied by each rotation.

    Unfinished.
    """
    R_mats = []

    length = amount_of_frames
    rotation_angles_deg = experiment_df["diabolo_red"]["Rotation"].iloc[start_index:(start_index+amount_of_frames)].values
    positions = experiment_df["diabolo_red"]["Position"].iloc[start_index:(start_index+amount_of_frames)].values
    for i in range(len(rotation_angles_deg)):
        # Get the orientations
        R_mats.append(tftf.quaternion_euler(rotation_angles_deg[i, :]))
        # TODO(felixvd): Fix matrix/quaternion

    ## Random Walk algorithm to find the rotation axis
    p = 0.01
    tries = 0
    residual = 100000

    rotation_axis_old = np.array([1,0,0,1])
    rotation_axis_new = np.array([1,0,0,1])
    
    while residual > 1e-6:
        tries += 1
        rotated_axes = []
        rotation_axis_new = copy.deepcopy(rotation_axis_old) + (np.random.rand(4)-0.5)*p
        rotation_axis_new[0:3] = rotation_axis_new[0:3]/np.linalg.norm(rotation_axis_new[0:3]) # Normalize
        rotation_axis_new[3] = 1.0
        # normalize the first three elements
        for i in range(len(R_mats)):
            # Get the rotated version of each axis
            rotated_axis = np.dot(R_mats[i],rotation_axis_new.transpose()) # TODO(felixvd): Fix matrix/quaternion
            rotated_axes.append(rotated_axis)

        rdf = pd.DataFrame(rotated_axes)
        residual_new = sum(rdf.std())
        if residual_new < residual:
            rotation_axis_old = copy.deepcopy(rotation_axis_new)
            residual = residual_new
            print("new residual = " + str(residual_new))
            tries = 0

        if tries > 100:
            print("reducing factor")
            p = p*0.1
            tries = 0
            if p < 1e-8:
                print("breaking out due to excessive tries")
                break

    print("The estimated rotation axis at start_index is: ", rotation_axis_old)
    return rotation_axis_old
    # Moving ~7 cm backwards along this axis should be close to the center of the diabolo.

In [70]:
def get_total_movement(experiment_df, start_index, amount_of_frames):
    """Return sum of distance between positions. Ignores rotation.
    Dataframe needs to be the one of the diabolo (with single header ("Position", "Rotation"))
    """
    xyz_diff_vals = experiment_df["Position"].iloc[start_index:(start_index+amount_of_frames)].diff().values
    distances = np.linalg.norm(xyz_diff_vals, axis=1)
    return np.sum(distances[1:])

In [76]:
def get_rotation_speeds(experiment_df, start_index, amount_of_frames):
    quat_vals = experiment_df["Rotation"].iloc[start_index:(start_index+amount_of_frames)].values
    speeds = [0]  # The first element of quat_vals is NaN, so we skip it.
    for i in range(1, len(quat_vals)):
        speeds.append(angle_between_quaternions(quat_vals[i-1,:], quat_vals[i,:]))
    return speeds

In [ ]:
## ======================================== Notes

In [154]:
# Read in experiment file
experiment_directory = "../experiments/recordings/2020-04-05"
f = pd.read_csv(os.path.join(experiment_directory, "Take_diabolo_red.csv"), header=[2,4,5])

# The data is noisy, so smoothing the position values (but not the rotation values)
# should help.
f2 = f.copy()
f_smoothed = f2.rolling(10).mean()
f2.loc[:, ("diabolo_red","Position")] = f_smoothed
f2.loc[:, ("stick_left","Position")] = f_smoothed
f2.loc[:, ("stick_right","Position")] = f_smoothed

In [181]:
# Smooth files and write output to separate file
f = pd.read_csv(os.path.join(experiment_directory, "Take_diabolo_red.csv"), header=[2,4,5])
f2 = f.copy()
f_smoothed = f.rolling(15).mean()   # 5 frames is still wobbly
f2.loc[:, ("diabolo_red","Position")] = f_smoothed
f2.loc[:, ("stick_left","Position")] = f_smoothed
f2.loc[:, ("stick_right","Position")] = f_smoothed
c.experiment_df = f2

c.diabolo_data = c.experiment_df["diabolo_red"]
c.stick_left_data = c.experiment_df["stick_left"]
c.stick_right_data = c.experiment_df["stick_right"]

c.calculate_and_apply_offsets(experiment_directory)
c.write_offset_files(filename="../experiments/output/2020-04-05/Take_diabolo_red_smoothed.csv")

### Smoothing rotations did not work well at all. The rotation axis should be considered instead.
# f2.loc[:, ("diabolo_red","Rotation")] = f_smoothed
# f2.loc[:, ("stick_left","Rotation")] = f_smoothed
# f2.loc[:, ("stick_right","Rotation")] = f_smoothed
# c.experiment_df = f2

# c.diabolo_data = c.experiment_df["diabolo_red"]
# c.stick_left_data = c.experiment_df["stick_left"]
# c.stick_right_data = c.experiment_df["stick_right"]

# c.calculate_and_apply_offsets(experiment_directory)
# c.write_offset_files(filename="../experiments/output/2020-04-05/Take_diabolo_red_smoothed_with_rotation.csv")

Calculating offsets
Applying offsets
Done applying offsets


In [ ]:
experiment_directory = "../experiments/recordings/2020-04-05"
f = pd.read_csv(os.path.join(experiment_directory, "Take_diabolo_red.csv"), header=[2,4,5])

f2 = f.copy()
f_smoothed = f.rolling(10).mean()
c.experiment_df = f_smoothed

c.diabolo_data = self.experiment_df["diabolo_red"]
c.stick_left_data = self.experiment_df["stick_left"]
c.stick_right_data = self.experiment_df["stick_right"]

In [182]:
import time
t = time.time()
for i in range(100):
    a = f.loc[15, ("diabolo_red", "Position")]
    f.at[15, ("diabolo_red", "Position", "X")] = 1.0
    f.at[15, ("diabolo_red", "Position", "Y")] = 2.0
    f.at[15, ("diabolo_red", "Position", "Z")] = 3.0

print("Elapsed time:", time.time() - t)
    
f.loc[14:16, ("diabolo_red", "Position")]

Elapsed time: 0.0455937385559082


,X,Y,Z
14,-0.130022,0.022345,0.376146
15,1.000000,2.000000,3.000000
16,-0.132009,0.031569,0.398590
